In [1]:
import os,sys
import xarray as xr
import numpy as np

In [2]:
#Get 1x1 degree grid
with xr.open_dataset('../rawdata/rsl_85_2090.nc') as ds:
    lon = ds['x'].values
    lat = ds['y'].values
    slr_md = ds['slr_md'].values

#Get fingerprints
with xr.open_dataset('../rawdata/fp_uniform/AIS.nc') as ds:
    lon05  = ds['x'].values
    lat05  = ds['y'].values
    fp05_a = ds['rsl'].values
with xr.open_dataset('../rawdata/fp_uniform/GrIS.nc') as ds:
    fp05_gr = ds['rsl'].values
fp05_gl = np.zeros((19,len(lat05),len(lon05)))
for r in range(1,20):
    with xr.open_dataset(f'../rawdata/fp_uniform/RGI_{r}.nc') as ds:
        fp05_gl[r-1,:,:] = ds['rsl'].values

#Convert to 1x1 degree grid
fp_a  = np.zeros((len(lat),len(lon)))
fp_gr = np.zeros((len(lat),len(lon)))
fp_gl = np.zeros((19,len(lat),len(lon)))

for i,ii in enumerate(lat):
    ig = np.argmin((ii-lat05)**2)
    for j,jj in enumerate(lon):
        jg = np.argmin((jj-lon05)**2)
        fp_a[i,j] = fp05_a[ig,jg]
        fp_gr[i,j] = fp05_gr[ig,jg]
        fp_gl[:,i,j] = fp05_gl[:,ig,jg]

fp_a[np.isnan(slr_md)] = np.nan
fp_gr[np.isnan(slr_md)] = np.nan
fp_gl[:,np.isnan(slr_md)] = np.nan

In [4]:
#Save
fp_a2 = xr.DataArray(fp_a,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
fp_gr2 = xr.DataArray(fp_gr,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
fp_gl2 = xr.DataArray(fp_gl,dims=('region','lat','lon'),coords={'region':np.arange(19),'lat':lat,'lon':lon})

ds = xr.Dataset({'fp_a':fp_a2,'fp_gr':fp_gr2,'fp_gl':fp_gl2})
ds.to_netcdf(f'../data/fingerprints.nc')
ds.close()